In [3]:
import itertools

def get_permutations(s):
    permutations = itertools.permutations(s)
    return [''.join(p) for p in permutations]

In [4]:
def readlines(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines

In [7]:
def assemble(filename, n=2):
    lines = readlines(filename)
    new_lines = []
    for line in lines:
        new_lines.append(line.rstrip('\n'))
    lines = new_lines

    sets = []
    for word in lines:
        short = [word]
        for combo in get_permutations(word):
            if combo in lines and combo != word:
                short.append(combo)
                lines.remove(combo)
        sets.append(short)

    new_sets = []
    for set in sets:
        if len(set) >= n:
            new_sets.append(set)

    return new_sets

In [11]:
def get_trees(root = [0], n = 0, max = 6):
    trees = []
    
    def generate(root, n, max):
        # if n >= max -1:
        #     return None
        # if root[-1] >= (2**max)-1:
        #     return None
        if len(root) == max:
            trees.append(root)
            return None
        if len(root) > max:
            return None
        if n >= len(root):
            return None
        
        left = (root[n] * 2) + 1
        lefter = root + [left]
        right = (root[n] * 2) + 2
        righter = root + [right]
        doubler = lefter + [right]
        
        generate(lefter, n+1, max)
        generate(righter, n+1, max)
        generate(doubler, n+1, max)
        generate(root, n+1, max)

    generate(root, n, max)

    return trees

In [32]:
def breadth_first(tree):
    result = ''
    for char in tree.values():
        result += char
    return result

def pre_order(tree):
    word = []
    def traverse(key, tree):
        if key in tree.keys():
            word.append(tree[key])
            traverse((2*key)+1, tree)
            traverse((2*key)+2, tree)
    traverse(0, tree)
    return ''.join(word)

def in_order(tree):
    word = []
    def traverse(key, tree):
        if key in tree.keys():
            traverse((2*key)+1, tree)
            word.append(tree[key])
            traverse((2*key)+2, tree)
    traverse(0, tree)
    return ''.join(word)

def post_order(tree):
    word = []
    def traverse(key, tree):
        if key not in tree.keys():
            return
        traverse((2*key)+1, tree)
        traverse((2*key)+2, tree)
        word.append(tree[key])
    traverse(0, tree)
    return ''.join(word)

def find_words(n):
    mapper = {4:'four', 5:'five', 6:'six', 7:'seven', 8:'eight'}
    num = mapper[n]
    with open(f'words_{num}.txt', 'r') as file:
        words = file.read().split()
    print(f'words_{num}.txt')
    return words

def make_tree(combo, array):
    tree = {}
    for num, char in zip(array, combo):
        tree[num] = char
    return tree


def check_tree(tree, n, words):
    #words = find_words(len(combo))
    #tree = make_tree(combo, array)

    count = 0
    pre = pre_order(tree)
    ino = in_order(tree)
    post = post_order(tree)
    bf = breadth_first(tree)
    true_trav = []
    used = []
    for trav in [pre, ino, post, bf]:
        if trav in words and trav not in used:
            count += 1
            used.append(trav)
            true_trav.append(trav)
    if count >= n:
        return tree, true_trav
    else:
        return None




In [33]:
def tree2tuple(tree):
    tup = tuple(list(tree.items()))
    return tup

def tuple2tree(trees):
    solutions = []
    for tree in trees:
        solutions.append(dict(tree))
    return solutions
    

In [34]:
def check_word(s,n, solutions, words, trees):
    combos = get_permutations(s)
    good_words = set()
    tree_set = set()
    for combo in combos:
        for t in trees:
            tree = make_tree(combo, t)
            result = check_tree(tree, n, words)
            if result: 
                tree_set.add(tree2tuple(tree))
                good_words.update(result[1])
    results = tuple2tree(tree_set)
    solutions[tuple(good_words)] = results


In [ ]:
from collections import defaultdict

def get_solutions(n, words, trees):
    solutions = defaultdict(list)
    for combo in words:
        s = combo[0]
        check_word(s, n , solutions, words = combo, trees = trees)
    return solutions

n = 2
words = assemble('resources/words_six.txt', 1)
trees = get_trees([0], 0, max = 6)

solutions_6 = get_solutions(n, words, trees)

In [18]:
import json

def tuple2string(tup):
    i = 0
    s = ''
    for item in tup:
        if i > 0:
            s += '/'
        s+=item
        i+=1
    return s

def solutions2json(solutions, n):
    nsolutions = {}
    for key, value in solutions.items():
        nsolutions[tuple2string(key)] = value
    
    with open(f'solutions_{n}.json', 'w') as json_file:
        json.dump(nsolutions, json_file, indent=4)

In [53]:
n = 2
words = assemble('resources/words_five.txt', 1)
trees = get_trees([0], 0, max = 5)

solutions_5 = get_solutions(n, words, trees)

In [ ]:
n = 2
words = assemble('resources/words_seven.txt', 1)
trees = trees = get_trees([0], 0, max = 7)

solutions_7 = get_solutions(n, words, trees)

In [26]:
import sys
sys.path.append('/Users/jackson/Documents/WordTree/WordTree')

In [36]:
import numpy as np
from collections import deque


def tree_height(tree):
    nodes = list(tree.keys())
    max = nodes[-1] + 1
    height = np.floor(np.log2(max))+1
    return int(height)


def num_spaces_for_level(height, level):
    return (2**(height-level-1))-1


def num_nodes_in_level(level):
    if level == 0:
        return 1
    else:
        return (2 * level)


def print_spaces(quantity):
    quantity = int(quantity)
    s = ''
    for i in range(quantity):
        s += ' '
    return s


def get_left(tup, tree):
    node, value = tup
    left_child = (node * 2) + 1
    if left_child in tree.keys():
        #print(f'get_left--parent: {tup}, left_child: {left_child}, {tree[left_child]}')
        return (left_child, tree[left_child])
    else:
        return None
    

def get_right(tup, tree):
    node, value = tup
    right_child = (node * 2) + 2
    if right_child in tree.keys():
        #print(f'get_right--parent: {tup}, right_child: {right_child}, {tree[right_child]}')
        return (right_child, tree[right_child])
    else:
        return None


def pretty_print_tree(tree):
    result = ''
    traversal = deque()
    traversal.append(list(tree.items())[0])
    #print(traversal)
    height = tree_height(tree)
    #print(f'height is {height}')
    for l in range(height):
        spaces_before = num_spaces_for_level(height, l)
        spaces_in_between = (2*spaces_before)+1
        #print(f'level: {l}, spaces between: {spaces_in_between}')
        result += print_spaces(spaces_before)

        nodes = num_nodes_in_level(l)
        #print(f'nodes: {nodes}')
        print_spaces_before = False
        for ni in range(nodes):
            if print_spaces_before:
                result += print_spaces(spaces_in_between)
            print_spaces_before = True
            node = traversal.popleft()

            if not node:
                result += ' '
                traversal.append(None)
                traversal.append(None)
            else:
                spot, value = node
                result += f'{value}'
                traversal.append(get_left(node, tree))
                traversal.append(get_right(node, tree))
        if (l != height -1):
            result += "\n"
    result += "\n"
    return(result)

In [89]:
def tuple2string(tup):
    i = 0
    s = ''
    for item in tup:
        if i > 0:
            s += '/'
        s+=item
        i+=1
    return s

nsolutions_5 = {}

for key, value in solutions_5.items():
    nsolutions_5[tuple2string(key)] = value

In [91]:
import json

with open('solutions_5.json', 'w') as json_file:
    json.dump(nsolutions_5, json_file, indent=4)

In [92]:
trees_5 = {}
for key, value in nsolutions_5.items():
    final_list = []
    for tree in value:
        final_list.append(pretty_print_tree(tree))
    trees_5[key] = final_list

In [94]:
import json

with open('trees_5.json', 'w') as json_file:
    json.dump(trees_5, json_file, indent=4)

In [22]:
def solve(word, trees = False):
    n = len(word)
    with open(f'solutions_{n}.json', 'r') as file:
        solutions = json.load(file)

    for key, item in solutions:
        if word in key:
            for i in item:
                print(i)

In [ ]:
import json

# Load JSON data from a file
with open('solutions_6.json', 'r') as file:
    data = json.load(file)

# Print all keys in the JSON data
def print_keys(json_data, parent_key=''):
    if isinstance(json_data, dict):
        for key in json_data:
            full_key = f"{parent_key}.{key}" if parent_key else key
            print(full_key)
            print_keys(json_data[key], full_key)
    elif isinstance(json_data, list):
        for index, item in enumerate(json_data):
            full_key = f"{parent_key}[{index}]"
            print_keys(item, full_key)

print_keys(data)


In [41]:
def str2int(tree):
    ntree = {}
    for key, value in tree.items():
        ntree[int(key)] = value
    return ntree

In [63]:
import json

def solve(word, trees=True):
    n = len(word)
    with open(f'solutions_{n}.json', 'r') as file:
        solutions = json.load(file)

    result = {}
    for key, value in solutions.items():
        if word in key:
            result[key] = value

    if result:
        for key, value in result.items():
            print(f"solved anagrams: {key}")
            print("List of possible trees:")
            for item in value:
                if trees:
                    nitem = str2int(item)
                    print(pretty_print_tree(nitem))
                    print('-'*50)
                else:
                    print(item)
    else:
        print(f"No solutions found for word '{word}'")


In [47]:
word = 'abides'
solve(word, trees=False)

solved anagrams: abides/biased
List of possible trees:
{'0': 'a', '1': 'b', '2': 'd', '4': 'i', '5': 'e', '11': 's'}
{'0': 'd', '1': 'a', '2': 'e', '4': 'i', '6': 's', '9': 'b'}


In [64]:
word = 'abides'
solve(word)

solved anagrams: abides/biased
List of possible trees:
       a
   b       d
     i   e    
        s  

--------------------------------------------------
       d
   a       e
     i       s
    b      

--------------------------------------------------
